In [262]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def to_1D(series):
    return pd.Series([x for _list in series for x in _list])

In [66]:
# This download may take up to 5 minutes
url = "https://drive.google.com/uc?export=download&id=1ziexNQvec0ilylv-U73nJoKuWenKCk5v&confirm=t&uuid=99001e73-0e6c-4793-b4ce-ac3033cbe37e&at=AHV7M3d_G0FOnB8Mm8zWlcZOSdsV:1670106934976"
users = pd.read_csv(url)
users.head()

KeyboardInterrupt: 

In [152]:
def strDateToNum(date):
    date_list = date.split(" to ")
    newDate = date_list[0]
    year = int(newDate[-4:])
    return 2023 - year

In [156]:
def strDurToNum(dur):
    dur_list = dur.split(" hr.")
    if "sec." in dur:
        return 1
    if len(dur_list) > 1:
        hr = int(dur_list[0])
        if dur_list[1] == "" or dur_list[1] == " per ep.":
            return hr * 60
        mins = dur_list[1][1:]
        try:
            mins = int(mins.split(" min.")[0])
        except ValueError:
            print(dur)
        return mins + 60 * hr
    else:
        dur2 = dur_list[0].split(" min.")
        return int(dur2[0])

In [254]:
def preprocess(df):
    df = df.replace('Unknown', np.nan)
    df["Score"] = df["Score"].ffill()

    for i, row in df.iterrows():
        if not isinstance(row["Aired"], str):
            continue
        df.at[i, "Aired"] = strDateToNum(row["Aired"])
    df["Aired"] = df["Aired"].ffill()

    for i, row in df.iterrows():
        if pd.isna(row["Episodes"]):
            if row["Type"] == "Movie":
                df.at[i, "Episodes"] = 1
            else:
                df.at[i, "Episodes"] = row["Aired"] * 96
    df["Episodes"] = df["Episodes"].ffill()

    for i, row in df.iterrows():
        if not isinstance(row["Duration"], str):
            continue
        df.at[i, "Duration"] = strDurToNum(row["Duration"])
    df["Duration"] = df["Duration"].ffill()

    df["Rating"] = df["Rating"].fillna("G - All Ages")
    df["Source"] = df["Source"].fillna("")

    columns_to_drop = ["English name", "Japanese name", "Premiered", "Ranked", "Score-10", "Score-9", 
                    "Score-8", "Score-7", "Score-6", "Score-5", "Score-4", "Score-3", "Score-2", 
                    "Score-1", "Licensors", "Producers", "Studios"]
    df = df.drop(columns_to_drop, axis=1)

    for i, row in df.iterrows():
        if pd.isna(row["Type"]):
            if row["Episodes"] == 1:
                df.at[i, "Type"] = "Movie"
            else:
                df.at[i, "Type"] = "TV"
    
    df["Genres"] = df["Genres"].fillna("")
    count2 = []

    categorical_columns = ["Genres", "Source", "Type", "Rating"]
    for col in categorical_columns:
        print(f"Processing {col}")
        for i, row in df.iterrows():
            df.at[i, col] = row[col].split(", ")

        counts = to_1D(df[col]).value_counts()
        if col == "Genres":
            count2 = counts.index

        for label in counts.index:
            for i,row in df.iterrows():
                if label in row[col]:
                    df.at[i, label] = 1
                else:
                    df.at[i, label] = 0

    columns_to_drop = ["Genres", "Name", "Source", "Type", "Rating"]
    df = df.drop(columns_to_drop, axis=1)

    return df, count2

In [221]:
anime["Genres"].value_counts()

[Hentai]                                                  969
[Music]                                                   790
[Comedy]                                                  668
[Kids]                                                    413
[Kids, Music]                                             234
                                                         ... 
[Action, Comedy, Samurai]                                   1
[Action, Sci-Fi, Police, Psychological, Mecha, Seinen]      1
[Action, Comedy, Hentai]                                    1
[Fantasy, Kids, Magic, Romance, Shoujo, Super Power]        1
[Adventure, Mystery, Supernatural]                          1
Name: Genres, Length: 5034, dtype: int64

In [256]:
anime, counts = preprocess(anime_old)

Processing Genres
Processing Source
Processing Type
Processing Rating


In [257]:
anime.head()

,MAL_ID,Score,Episodes,Aired,Duration,Popularity,Members,Favorites,Watching,Completed,...,OVA,Movie,Special,ONA,G - All Ages,PG-13 - Teens 13 or older,PG - Children,Rx - Hentai,R - 17+ (violence & profanity),R+ - Mild Nudity
0,1,8.78,26,25,24,39,1251960,61971,105808,718161,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,5,8.39,1,22,115,518,273145,1174,4143,208333,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,6,8.24,26,25,24,201,558913,12944,29113,343492,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,7,7.27,26,21,25,1467,94683,587,4300,46165,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,8,6.98,52,19,23,4369,13224,18,642,7314,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [264]:
url = "anime.csv"
anime_old = pd.read_csv(url)
anime_old.head()

,MAL_ID,Name,Score,Genres,English name,Japanese name,Type,Episodes,Aired,Premiered,...,Score-10,Score-9,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Cowboy Bebop,カウボーイビバップ,TV,26,"Apr 3, 1998 to Apr 24, 1999",Spring 1998,...,229170.0,182126.0,131625.0,62330.0,20688.0,8904.0,3184.0,1357.0,741.0,1580.0
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space",Cowboy Bebop:The Movie,カウボーイビバップ 天国の扉,Movie,1,"Sep 1, 2001",Unknown,...,30043.0,49201.0,49505.0,22632.0,5805.0,1877.0,577.0,221.0,109.0,379.0
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",Trigun,トライガン,TV,26,"Apr 1, 1998 to Sep 30, 1998",Spring 1998,...,50229.0,75651.0,86142.0,49432.0,15376.0,5838.0,1965.0,664.0,316.0,533.0
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),TV,26,"Jul 2, 2002 to Dec 24, 2002",Summer 2002,...,2182.0,4806.0,10128.0,11618.0,5709.0,2920.0,1083.0,353.0,164.0,131.0
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",Beet the Vandel Buster,冒険王ビィト,TV,52,"Sep 30, 2004 to Sep 29, 2005",Fall 2004,...,312.0,529.0,1242.0,1713.0,1068.0,634.0,265.0,83.0,50.0,27.0


In [237]:
anime = anime.drop(["Studios"], axis=1)

In [234]:
anime.isna().sum()

MAL_ID       0
Name         0
Score        0
Genres       0
Type         0
            ..
Josei        0
Shoujo Ai    0
             0
Yaoi         0
Yuri         0
Length: 63, dtype: int64

In [238]:
anime["Source"].value_counts()

Original         5215
Manga            3825
                 3567
Visual novel      993
Game              880
Light novel       768
Other             597
Novel             510
Music             317
4-koma manga      288
Web manga         252
Picture book      147
Book              112
Card game          64
Digital manga      15
Radio              12
Name: Source, dtype: int64

In [199]:
for i, row in anime.iterrows():
    anime.at[i, "Genres"] = row["Genres"].split(", ")

In [222]:
anime["Source"].value_counts()

                                               13616
Funimation                                       883
Sentai Filmworks                                 652
Media Blasters                                   206
Discotek Media                                   194
                                               ...  
Bandai                                             1
Nozomi Entertainment, Kadokawa Pictures USA        1
Sentai Filmworks, Crunchyroll                      1
ADV Films, Funimation, OLM Digital                 1
iQIYI                                              1
Name: Licensors, Length: 231, dtype: int64

In [223]:
for i, row in anime.iterrows():
    anime.at[i, "Licensors"] = row["Licensors"].split(", ")

In [228]:
to_1D(anime["Genres"])

0               Action
1            Adventure
2               Comedy
3                Drama
4               Sci-Fi
             ...      
50256        Adventure
50257    Slice of Life
50258           Comedy
50259           Action
50260          Fantasy
Length: 50261, dtype: object

In [231]:
to_1D(anime["Studio"]).nunique()

KeyError: 'Studio'

In [200]:
anime["Genres"]

0        [Action, Adventure, Comedy, Drama, Sci-Fi, Space]
1                  [Action, Drama, Mystery, Sci-Fi, Space]
2        [Action, Sci-Fi, Adventure, Comedy, Drama, Sho...
3        [Action, Mystery, Police, Supernatural, Drama,...
4              [Adventure, Fantasy, Shounen, Supernatural]
                               ...                        
17557                   [Adventure, Mystery, Supernatural]
17558                       [Comedy, Horror, Supernatural]
17559    [Mystery, Dementia, Horror, Psychological, Sup...
17560                   [Adventure, Slice of Life, Comedy]
17561                                    [Action, Fantasy]
Name: Genres, Length: 17562, dtype: object

In [204]:
def to_1D(series):
 return pd.Series([x for _list in series for x in _list])

counts = to_1D(anime["Genres"]).value_counts()
counts

Comedy           6029
Action           3888
Fantasy          3285
Adventure        2957
Kids             2665
Drama            2619
Sci-Fi           2583
Music            2244
Shounen          2003
Slice of Life    1914
Romance          1899
School           1642
Supernatural     1479
Hentai           1348
Historical       1144
Mecha            1101
Magic            1081
Seinen            830
Ecchi             767
Mystery           727
Sports            713
Shoujo            688
Parody            660
Super Power       632
Military          576
Dementia          512
Demons            501
Space             495
Horror            462
Martial Arts      425
Harem             399
Game              386
Psychological     345
Police            247
Samurai           202
Vampire           136
Cars              133
Thriller          131
Shounen Ai        100
Josei              97
Shoujo Ai          79
                   63
Yaoi               42
Yuri               32
dtype: int64

In [208]:
for label in counts.index:
    for i,row in anime.iterrows():
        if label in row["Genres"]:
            anime.at[i, label] = 1
        else:
            anime.at[i, label] = 0

anime.head()

,MAL_ID,Name,Score,Genres,Type,Episodes,Aired,Producers,Licensors,Studios,...,Samurai,Vampire,Cars,Thriller,Shounen Ai,Josei,Shoujo Ai,,Yaoi,Yuri
0,1,Cowboy Bebop,8.78,"[Action, Adventure, Comedy, Drama, Sci-Fi, Space]",TV,26,25,Bandai Visual,"Funimation, Bandai Entertainment",Sunrise,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"[Action, Drama, Mystery, Sci-Fi, Space]",Movie,1,22,"Sunrise, Bandai Visual",Sony Pictures Entertainment,Bones,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,6,Trigun,8.24,"[Action, Sci-Fi, Adventure, Comedy, Drama, Sho...",TV,26,25,Victor Entertainment,"Funimation, Geneon Entertainment USA",Madhouse,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,7,Witch Hunter Robin,7.27,"[Action, Mystery, Police, Supernatural, Drama,...",TV,26,21,"TV Tokyo, Bandai Visual, Dentsu, Victor Entert...","Funimation, Bandai Entertainment",Sunrise,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,8,Bouken Ou Beet,6.98,"[Adventure, Fantasy, Shounen, Supernatural]",TV,52,19,"TV Tokyo, Dentsu",,Toei Animation,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [210]:
anime["Licensors"].value_counts()

[]                                               13616
[Funimation]                                       883
[Sentai Filmworks]                                 652
[Media Blasters]                                   206
[Discotek Media]                                   194
                                                 ...  
[Bandai]                                             1
[Nozomi Entertainment, Kadokawa Pictures USA]        1
[Sentai Filmworks, Crunchyroll]                      1
[ADV Films, Funimation, OLM Digital]                 1
[iQIYI]                                              1
Name: Licensors, Length: 231, dtype: int64

In [243]:
anime["Rating"].value_counts()

G - All Ages                      6470
PG-13 - Teens 13 or older         6132
PG - Children                     1461
Rx - Hentai                       1345
R - 17+ (violence & profanity)    1157
R+ - Mild Nudity                   997
Name: Rating, dtype: int64

In [244]:
categorical_columns = ["Type", "Rating"]

for col in categorical_columns:
    print(f"Preprocessing {col}")
    for i, row in anime.iterrows():
        anime.at[i, col] = row[col].split(", ")

    counts = to_1D(anime[col]).value_counts()

    for label in counts.index:
        for i,row in anime.iterrows():
            if label in row[col]:
                anime.at[i, label] = 1
            else:
                anime.at[i, label] = 0

Preprocessing Type
Preprocessing Rating


In [248]:
columns_to_drop = ["Genres", "Name", "Source", "Type", "Rating"]
anime = anime.drop(columns_to_drop, axis=1)

In [ ]:
anime.head()

In [193]:
anime["Genres"] = anime["Genres"].fillna("")

In [188]:
anime["Type"].value_counts()

TV         4996
OVA        3894
Movie      3041
Special    2218
ONA        1907
Music      1469
Name: Type, dtype: int64

In [189]:
for i, row in anime.iterrows():
    if pd.isna(row["Type"]):
        if row["Episodes"] == 1:
            anime.at[i, "Type"] = "Movie"
        else:
            anime.at[i, "Type"] = "TV"

In [184]:
columns_to_drop = ["English name", "Japanese name", "Premiered", "Ranked", "Score-10", "Score-9", 
                    "Score-8", "Score-7", "Score-6", "Score-5", "Score-4", "Score-3", "Score-2", "Score-1"]

In [185]:
anime = anime.drop(columns_to_drop, axis=1)

In [178]:
anime["Licensors"] = anime["Licensors"].fillna("")
anime["Producers"] = anime["Producers"].fillna("")
anime["Source"] = anime["Source"].fillna("")
anime["Studios"] = anime["Studios"].fillna("")

In [179]:
anime["Producers"] = anime["Producers"].fillna("")

In [180]:
anime["Source"] = anime["Source"].fillna("")

In [181]:
anime["Studios"] = anime["Studios"].fillna("")

In [167]:
anime["Producers"].value_counts()

NHK                                                                                623
Pink Pineapple                                                                     184
Sanrio                                                                             160
Bandai Visual                                                                      117
Fuji TV                                                                            112
                                                                                  ... 
Geneon Universal Entertainment, Warner Bros. Japan, ASCII Media Works, Showgate      1
Dentsu, Fuji TV, Asmik Ace, Sony Music Entertainment, Bungeishunjuu                  1
Shogakukan-Shueisha Productions, NHK, Animation 21                                   1
Shogakukan-Shueisha Productions, Bandai                                              1
Bushiroad, Bit grooove promotion                                                     1
Name: Producers, Length: 3782, dtype: int64

In [172]:
anime["Rating"].isna().sum()

0

In [171]:
anime["Rating"] = anime["Rating"].fillna("G - All Ages")

In [173]:
anime["Rating"].value_counts()

G - All Ages                      6470
PG-13 - Teens 13 or older         6132
PG - Children                     1461
Rx - Hentai                       1345
R - 17+ (violence & profanity)    1157
R+ - Mild Nudity                   997
Name: Rating, dtype: int64

In [165]:
anime["Licensors"].value_counts()

Funimation                            883
Sentai Filmworks                      652
Media Blasters                        206
Discotek Media                        194
Aniplex of America                    177
                                     ... 
Bandai                                  1
Brain's Base, Discotek Media            1
Sentai Filmworks, Crunchyroll           1
ADV Films, Funimation, OLM Digital      1
iQIYI                                   1
Name: Licensors, Length: 230, dtype: int64

In [127]:
anime = anime.replace('Unknown', np.nan)

In [72]:
anime.isna().sum()

MAL_ID               0
Name                 0
Score             5141
Genres              63
English name     10565
Japanese name       48
Type                37
Episodes           516
Aired              309
Premiered        12817
Producers         7794
Licensors        13616
Studios           7079
Source            3567
Duration           555
Rating             688
Ranked            1762
Popularity           0
Members              0
Favorites            0
Watching             0
Completed            0
On-Hold              0
Dropped              0
Plan to Watch        0
Score-10           437
Score-9           3167
Score-8           1371
Score-7            503
Score-6            511
Score-5            584
Score-4            977
Score-3           1307
Score-2           1597
Score-1            459
dtype: int64

In [73]:
anime["Score"].isna().sum()

5141

In [128]:
anime["Score"] = anime["Score"].ffill()

In [75]:
anime["Score"].isna().sum()

0

In [76]:
temp_var = anime[anime["Episodes"].isna()]["Aired"]

In [82]:
print("sdfljs 2940"[4:])

js 2940


In [101]:
anime["Aired"].isna().sum()

309

In [251]:
import seaborn as sb
import matplotlib.pyplot as plt

In [258]:
counts

Index(['Comedy', 'Action', 'Fantasy', 'Adventure', 'Kids', 'Drama', 'Sci-Fi',
       'Music', 'Shounen', 'Slice of Life', 'Romance', 'School',
       'Supernatural', 'Hentai', 'Historical', 'Mecha', 'Magic', 'Seinen',
       'Ecchi', 'Mystery', 'Sports', 'Shoujo', 'Parody', 'Super Power',
       'Military', 'Dementia', 'Demons', 'Space', 'Horror', 'Martial Arts',
       'Harem', 'Game', 'Psychological', 'Police', 'Samurai', 'Vampire',
       'Cars', 'Thriller', 'Shounen Ai', 'Josei', 'Shoujo Ai', '', 'Yaoi',
       'Yuri'],
      dtype='object')

In [259]:
#Using Pearson Correlation
plt.figure(figsize=(150,75))
cor = anime[counts].corr()
sb.heatmap(cor, annot=True, cmap=plt.cm.Reds)
plt.savefig('corr.png')

In [125]:
anime = preprocess(anime)

TypeError: 'DataFrame' object is not callable

In [129]:
for i, row in anime.iterrows():
    if not isinstance(row["Aired"], str):
        continue
    #print(strDateToNum(row["Aired"]))
    anime.at[i, "Aired"] = strDateToNum(row["Aired"])


In [104]:
anime["Aired"].value_counts()

6     922
7     897
5     882
9     852
8     792
     ... 
70      2
72      2
86      2
78      1
79      1
Name: Aired, Length: 101, dtype: int64

In [130]:
anime["Aired"] = anime["Aired"].ffill()

In [106]:
anime["Aired"].isna().sum()

0

In [112]:
anime["Episodes"].isna().sum()

516

In [131]:
for i, row in anime.iterrows():
    if pd.isna(row["Episodes"]):
        if row["Type"] == "Movie":
            anime.at[i, "Episodes"] = 1
        else:
            anime.at[i, "Episodes"] = row["Aired"] * 96

In [115]:
anime["Episodes"].isna().sum()

0

In [121]:
anime["Duration"].isna().sum()

555

In [133]:
val = "sdfkjsldfji"
print(val[1:])

dfkjsldfji


In [141]:
for i, row in anime.iterrows():
    if not isinstance(row["Duration"], str):
        continue
    #print(strDateToNum(row["Aired"]))
    anime.at[i, "Duration"] = strDurToNum(row["Duration"])

In [143]:
anime["Duration"].value_counts()

24     2014
23     1320
30     1149
25     1058
2      1049
       ... 
137       1
152       1
167       1
132       1
148       1
Name: Duration, Length: 147, dtype: int64

In [142]:
anime["Duration"].isna().sum()

742

In [111]:
sorted(anime["Episodes"].value_counts())

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 5,
 5,
 5,
 5,
 5,
 5,
 6,
 6,
 6,
 7,
 7,
 7,
 7,
 7,
 8,
 8,
 8,
 9,
 9,
 9,
 10,
 10,
 11,
 11,
 11,
 12,
 12,
 12,
 12,
 13,
 14,
 14,
 14,
 15,
 15,
 16,
 17,
 17,
 17,
 19,
 20,
 20,
 21,
 21,
 22,
 22,
 28,
 29,
 37,
 38,
 41,
 42,
 43,
 44,
 49,
 57,
 57,
 63,
 73,
 89,
 97,
 107,
 107,
 118,
 119,
 170,
 200,
 206,
 237,
 258,
 340,
 451,
 619,
 631,
 769,
 1368,
 1445,
 8381]

In [37]:
anime["Type"].value_counts()

TV         4996
OVA        3894
Movie      3041
Special    2218
ONA        1907
Music      1469
Name: Type, dtype: int64

In [263]:
cosine_sim = cosine_similarity(anime, anime)
print(cosine_sim)

[[1.         0.98665848 0.99885649 ... 0.22276135 0.02852815 0.00638178]
 [0.98665848 1.         0.99243379 ... 0.19954356 0.02600864 0.00621345]
 [0.99885649 0.99243379 1.         ... 0.21957371 0.0282044  0.00638677]
 ...
 [0.22276135 0.19954356 0.21957371 ... 1.         0.96562742 0.95759168]
 [0.02852815 0.02600864 0.0282044  ... 0.96562742 1.         0.99957009]
 [0.00638178 0.00621345 0.00638677 ... 0.95759168 0.99957009 1.        ]]


In [265]:
indices = pd.Series(anime_old["Name"])

def recommend(title, cosine_sim = cosine_sim):
    recommended_anime = []
    idx = indices[indices == title].index[0]
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)
    top_10_indices = list(score_series.iloc[1:11].index)
    
    for i in top_10_indices:
        recommended_anime.append(list(anime_old['Name'])[i])
        
    return np.vstack((recommended_anime,score_series.iloc[1:11].values)).T

recommend('Cowboy Bebop')

array([['Hajime no Ippo', '0.9999388578352284'],
       ['Great Teacher Onizuka', '0.9998540950930935'],
       ['Psycho-Pass', '0.9994889563929324'],
       ['Durarara!!', '0.999468874580756'],
       ['Higurashi no Naku Koro ni', '0.9994295411520977'],
       ['Slam Dunk', '0.9991979325407833'],
       ['Violet Evergarden', '0.9991661710328398'],
       ['Hellsing Ultimate', '0.9991656249647023'],
       ['Hyouka', '0.9991139450317719'],
       ['Rurouni Kenshin: Meiji Kenkaku Romantan', '0.9991085435409062']],
      dtype='<U39')

In [266]:
recommend('One Piece')

array([['Black Clover', '0.9942264096181637'],
       ['Boruto: Naruto Next Generations', '0.9868062737642042'],
       ['Jujutsu Kaisen (TV)', '0.9838745642749771'],
       ['Shingeki no Kyojin: The Final Season', '0.9834763780634284'],
       ['Detective Conan', '0.9820447438397646'],
       ['Horimiya', '0.9719130525262355'],
       ['Kaifuku Jutsushi no Yarinaoshi', '0.9707002055355529'],
       ['Mushoku Tensei: Isekai Ittara Honki Dasu', '0.9702766608920692'],
       ['Re:Zero kara Hajimeru Isekai Seikatsu 2nd Season Part 2',
        '0.9632608599859072'],
       ['Crayon Shin-chan', '0.9599636285754516']], dtype='<U55')

In [268]:
recommend('Naruto')

array([['Yu☆Gi☆Oh! Duel Monsters', '0.9993450157955739'],
       ['Dragon Ball', '0.9993340212541764'],
       ['Death Note', '0.9991147402220802'],
       ['Sword Art Online II', '0.9989871697312912'],
       ['Boku no Pico', '0.998752871553701'],
       ['Dragon Ball Kai', '0.9987076070633664'],
       ['Pokemon Diamond & Pearl', '0.9986349969896746'],
       ['Sword Art Online', '0.9985658790250881'],
       ['Shingeki no Kyojin', '0.9985248372085518'],
       ['Dragon Ball Z', '0.9983640887081232']], dtype='<U32')

In [267]:
recommend('Chainsaw Man')

array([['Goblin Slayer 2nd Season', '0.9999584278633603'],
       ['Maou Gakuin no Futekigousha: Shijou Saikyou no Maou no Shiso, Tensei shite Shison-tachi no Gakkou e',
        '0.9999055278525444'],
       ['Dungeon ni Deai wo Motomeru no wa Machigatteiru Darou ka IV',
        '0.9997687526602881'],
       ['Slime Taoshite 300-nen, Shiranai Uchi ni Level Max ni Nattemashita',
        '0.9997530955253202'],
       ['Hige wo Soru. Soshite Joshikousei wo Hirou.',
        '0.9996825364186337'],
       ['Free! (Movie)', '0.9995029618715787'],
       ['Shirobako Movie', '0.9981212901407539'],
       ['Boku no Hero Academia the Movie 3', '0.9979950863135179'],
       ['Fate/Grand Order: Shinsei Entaku Ryouiki Camelot 1 - Wandering; Agateram',
        '0.9975260689642438'],
       ['Mekakucity Reload', '0.9970180736656212']], dtype='<U99')

In [ ]:
tv_anime = anime[anime["Type"]=="TV"]
ova_anime = anime[anime["Type"]=="OVA"]
movie_anime = anime[anime["Type"]=="Movie"]
special_anime = anime[anime["Type"]=="Special"]
ona_anime = anime[anime["Type"]=="ONA"]
music_anime = anime[anime["Type"]=="Music"]